The data must now be shaped so that it can be used in the neural network. This is the same shape as the the turbofan i.e. multiple rows of sequential sensor data, in this case 60 cycles instead of 30. The data is saved in batches so that during training 1 file can easily be loaded and used as a single batch. Each batch file in this case contains 32 instances of model data(with each instance containing 60 cycles). The data is just sampled at a step rate of 60 since there is enough data as is.


First change directories and specify the folders needed to write to. 

In [1]:
import pandas as pd
import numpy as np
import glob,os
import pickle

dir = "D:\\WindData\\POD_Request_3541\\Project\\processed\\trimmed"

os.chdir(dir)

if not os.path.exists('final'):
    os.makedirs('final')
if not os.path.exists('final/data'):
        os.makedirs('final/data')


Note down the data files and label files which will be used.

In [2]:
labels = glob.glob("labels/*.csv")
dataSets = glob.glob("data/*.csv")
print(labels)



['labels\\0.csv', 'labels\\1.csv', 'labels\\10.csv', 'labels\\11.csv', 'labels\\12.csv', 'labels\\13.csv', 'labels\\14.csv', 'labels\\15.csv', 'labels\\16.csv', 'labels\\2.csv', 'labels\\3.csv', 'labels\\4.csv', 'labels\\5.csv', 'labels\\6.csv', 'labels\\7.csv', 'labels\\8.csv', 'labels\\9.csv']


Function for creating a new batch of data. Here the skiprows and nrows attributes are used to carve out specific parts of the data. When there is not enough data for a batch it returns an empty list

In [3]:
def pullBatch(numBatches, batchSize, depth, labelFile, dataFile):
    dataList = []
    avgList = []
    skip = numBatches * batchSize * depth
    for i in range(batchSize):
        frameData = pd.read_csv(dataFile, header=None, skiprows=skip, nrows=depth)
        if len(frameData) < depth:
            return [], 0
        else:
            avgData = pd.read_csv(labelFile, header=None, skiprows=skip, nrows=depth)
            avg = avgData[0].mean()
            avgList.append(avg)
            dataList.append(frameData.values.tolist())
            skip += depth
    return dataList, avgList
    

Batch size and number of cycles per instance is specified. The labels are also saved into one big list.

The loop takes every data/label file pair and makes as many batches batches as possible. The data from each batch is saved in its own file, while the labels are put into the label list. 

In [4]:
batchSize = 32
depth = 60
labelList = []
batchCount = 0

for label, data in zip(labels, dataSets):
    numBatches = 0
    batch, avgList = pullBatch(numBatches, batchSize, depth, label, data)
    while not len(batch)==0:
        npArray = np.array(batch)
        if not np.any(np.isnan(npArray)):
            labelList.append(avgList)
            np.save("final/data/" + str(batchCount) + ".npy", npArray)
        else:
            print("NaN found")
        numBatches+=1
        batchCount+=1
        batch, avgList = pullBatch(numBatches, batchSize, depth, label, data)
    print("Round done")

NaN found
NaN found
NaN found
NaN found
NaN found
Round done
NaN found
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
NaN found
NaN found
NaN found
NaN found
Round done
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
Round done
Round done
Round done
Round done


In [5]:
#frame = pd.DataFrame(labelList, )

Save the list of labels as a pickle so that it can easily be loaded for the model

In [6]:
pickle.dump(labelList, open("./final/labels.p", "wb"))